## Unit Testing in Python


### Code to be tested
For this example, we will be writing unit tests on a piece of code below. Make sure that you run the cell below before running the next code cells.

In [ ]:
import random

class Pokemon():
    def __init__(self, name, attacks={}):
        self.name = name
        self.attacks = attacks

    def add_attack(self, name, min, max):
        """
        Add an attack
        """
        self.attacks[name] = {"min": min, "max": max}

    def attack(self, name):
        """
        Returns a random damage given the attack name
        """
        return random.randrange(
            self.attacks[name]["min"],
            self.attacks[name]["max"]
        )

eevee = Pokemon("eevee")
eevee.add_attack("tail-whip", 5, 10)
print(eevee.attack("tail-whip"))

### Using unittest

Python already has a built-in testing framework called [unittest](https://docs.python.org/3/library/unittest.html). Which has been inspired by JUnit.

In its most basic form, unittests are grouped together into TestCases which each tests prefixed with *test*

```python
class TestGroup(unittest.TestCase):
    def setUp(self):
        ...
    def test_1(self):
        ...
    def test_N(self):
        ...
    def tearDown(self):
        ...
```

```setUp()``` and ```tearDown()``` are useful methods to implement as they allow you to define instructions that will be executed before and after each test method.

A sample implementation of the unittest for the Pokemon class is shown on a executable cell below:

In [ ]:
import unittest

class PokemonAttackMethods(unittest.TestCase):

    def setUp(self):
        self.pokemon = Pokemon("eevee")
        self.pokemon.add_attack("tail-whip", 5, 10)

    def test_pokemon_name(self):
        self.assertEqual(self.pokemon.name, "eevee")

    def test_pokemon_add_attack(self):
        self.assertEqual(self.pokemon.attacks['tail-whip'], {"min": 5, "max": 10})

    def test_get_attack_damage(self):
        damage = self.pokemon.attack("tail-whip")
        assert damage in range(5, 10)

    def tearDown(self):
        """
        Any cleanup you want here (ie. cleaning side effects)
        """


unittest.main(argv=[''], exit=False, verbosity=2)

### Using pytest

One of the most popular testing frameworks in python is [pytest](https://docs.pytest.org/en/stable/). It can be used not just for unit tests but also for functional tests as well and it has a huge collection of [plugins](http://plugincompat.herokuapp.com/) that extends its functionality. The following test automation examples on these setup of notebooks will use pytest moving forward.

In order to run the pytest script within jupyter. Please make sure that you execute the cell below in order to load the jupyter-pytest plugin.

In [ ]:
import ipytest
ipytest.autoconfig()

The code below is an implementation of the same unit test for the Pokemon class in pytest. Do take note that the first line is specific for jupyter only, remove if you are going to run it standalone.

In [ ]:
%%run_pytest[clean] -svv
import pytest

@pytest.fixture(scope="module")
def pokemon():
    p = Pokemon("eevee")
    p.add_attack("tail-whip", 5, 10)
    yield p # provide the fixture value
    ### Anything Beyond yield is the teardown code
    print("Tearing down...")

def test_pokemon_name(pokemon):
    assert pokemon.name == "eevee"

def test_pokemon_add_attack(pokemon):
    assert pokemon.attacks['tail-whip'] == {"min": 5, "max": 10}

def test_get_attack_damage(pokemon):
    damage = pokemon.attack("tail-whip")
    assert damage in range (5, 10)



pytest has a concept of **fixtures** that initialize tests. The fixture on the code above initializes and tears down the test run. Pytest has a set of useful built-in of useful fixtures [here](https://docs.pytest.org/en/stable/fixture.html) and each fixture can be created and destroyed based on their [scope](https://docs.pytest.org/en/stable/fixture.html#fixture-scopes). On the example above the *pokemon* fixture has a scope of *module* which is destroyed during teardown of the last test in the module.

### Generating Mock Data for tests

It is a testing best practice to use mock data to provide unpredictability and variance to tests. One of the most useful libraries for generating mock data is the **Faker** (https://pypi.org/project/Faker/) library. A sample code below generates random names, email, and phone numbers:

In [ ]:
from faker import Faker

fake = Faker()
for i in range(5):
    print("{} - {} - {}".format(fake.name(), fake.email(), fake.phone_number()))

Faker has these set of [standard](https://faker.readthedocs.io/en/latest/providers.html) and [community](https://faker.readthedocs.io/en/latest/communityproviders.html) providers.

Faker also plays along well with pytest. The sample code below improves upon the previous pytest by incorporating Faker to provide the mock inputs.

In [ ]:
%%run_pytest[clean] -svv

from faker import Faker

@pytest.fixture(scope="module")
def values():
    """
    Generate random values
    """
    fake = Faker()
    min_attack = fake.random_int(max=1000)
    max_attack = min_attack + fake.random_int(max=100)
    return {
        "name": fake.name(),
        "attack": fake.word(),
        "min": min_attack,
        "max": max_attack,
    }

@pytest.fixture(scope="module")
def pokemon(values):
    p = Pokemon(values['name'])
    p.add_attack(values['attack'], values['min'], values['max'])
    yield p # provides the fixture value
    ### Anything Beyond yield is the teardown code
    print("Tearing down...")

def test_pokemon_name(pokemon, values):
    assert values['name'] == pokemon.name

def test_pokemon_add_attack(pokemon, values):
    assert pokemon.attacks[values['attack']] == {"min": values['min'], "max": values['max']}

def test_get_attack_damage(pokemon, values):
    damage = pokemon.attack(values['attack'])
    assert damage in range (values['min'], values['max'])